# HyLands: modelando a evolução de paisagens e movimentos de massa

Este notebook oferece uma breve introdução e guia do usuário para simular deslizamentos profundos utilizando o componente **BedrockLandslider** do Landlab dentro do sistema HyLands. Ele foi escrito por Benjamin Campforts para acompanhar as seguintes publicações:

* Campforts, B., Shobe, C. M., Overeem, I., & Tucker, G. E. (2022). HyLands – A hybrid landscape evolution model. *Journal of Geophysical Research: Earth Surface*, 127(8). [doi: 10.1029/2022JF006745](https://doi.org/10.1029/2022JF006745)
* Campforts, B., Shobe, C. M., Steer, P., Vanmaercke, M., Lague, D., *et al.* (2020). HyLands model description. *Geoscientific Model Development*, 13, 3863–3889. [doi: 10.5194/gmd-13-3863-2020](https://doi.org/10.5194/gmd-13-3863-2020)

## Contexto do componente BedrockLandslider

O **BedrockLandslider** (um componente do modelo híbrido de evolução de paisagens – HyLands) simula, de forma estocástica, deslizamentos profundos que removem rocha matriz e solo, gerando o transporte de sedimentos derivados através de paisagens bidimensionais.

Este manual ensina como utilizar o BedrockLandslider isoladamente ou acoplado ao componente **SPACE** do Landlab. Para exemplos adicionais consulte o notebook do SPACE.

Pré‑requisitos: conhecimento básico de programação em Python e familiaridade com o framework Landlab (Hobley *et al.*, 2017, GMD; Barnhart *et al.*, 2020, eSurf).

## Descrição do modelo

### Parâmetros de entrada

- **angle_int_frict** $\psi$: Ângulo de atrito interno do material [m/m]. Padrão = 1.0
- **cohesion_eff** $c_{eff}$: Coesão efetiva do material [m L$^{-1}$ T$^{-2}$].
- **landslides_return_time** $t_{LS}$: Tempo de retorno para ocorrência estocástica de deslizamentos [anos]. Padrão = 1e5
- **rho_r** $\rho_r$: Densidade de massa da rocha [m L$^{-3}$].
- **fraction_fines_LS** $F_f$: Fração de finos permanentemente suspensíveis na rocha [-].
- **phi** $\phi$: Porosidade do sedimento [-].
- **max_pixelsize_landslide**: Tamanho máximo de um deslizamento em número de pixels. Padrão = 1e9
- **verbose_landslides**: Imprime o número de deslizamentos simulados por passo de tempo. Padrão = False
- **seed**: Semente para o gerador estocástico. Se não especificado, usa‑se 2021. Defina `None` para manter a semente atual.
- **landslides_on_boundary_nodes**: Permite que deslizamentos iniciem (nó crítico) e se estendam por nós de borda. Padrão = True
- **critical_sliding_nodes**: Lista de nós críticos nos quais novos deslizamentos precisam iniciar. Padrão = None

### Campos do modelo

Os campos abaixo são atualizados pelo componente nos nós do grid sempre que `run_one_step` é executado. Observação: alguns campos adicionais são criados por componentes de fluxo e pelo modelo SPACE; eles não estão listados aqui, mas são necessários para a execução completa do modelo.

- `soil__depth`, node, [m]: Espessura do solo (ou sedimento). A espessura é atualizada em todos os nós afetados por deslizamentos.
- `sediment__flux`, node, [m$^3$/ano]: Fluxo volumétrico de sedimento transportado; integra o sedimento removido e é usado para calcular taxas de deposição.
- `landslide__erosion`, node, [m]: Erosão total por deslizamentos em metros por nó.
- `landslide__deposition`, node, [m]: Deposição total de sedimento por deslizamentos em metros por nó.
- `landslide_sediment_point_source`, node, [m$^3$]: Volume de sedimento gerado no ponto onde o deslizamento se inicia, antes do cálculo do *run‑out*.

### Atributos do modelo

São criadas listas que armazenam estatísticas de deslizamentos e são reiniciadas a cada chamada de `landslide_erosion`:

- `landslides_size`: Tamanho (número de pixels) dos deslizamentos simulados.
- `landslides_volume`: Volume total dos deslizamentos simulados.
- `landslides_volume_sed`: Volume de sedimento erodido.
- `landslides_volume_bed`: Volume de rocha erodida.

## Etapas de um modelo BedrockLandslider

As etapas descritas a seguir referem‑se a um modelo BedrockLandslider **acoplado** ao componente SPACE. Para exemplos de um modelo desacoplado consulte a documentação em [http://landlab.github.io](http://landlab.github.io).

### Etapa 1: Importar as bibliotecas necessárias

Serão utilizados os componentes `BedrockLandslider` e `SPACE`, além do `PriorityFloodFlowRouter` para roteamento de fluxo em superfícies planas ou depressões.

In [ ]:
import copy

import matplotlib as mpl
import matplotlib.pyplot as plt  # For plotting results; optional
import numpy as np

from landlab import RasterModelGrid  # Grid utility
from landlab import imshow_grid, imshowhs_grid  # For plotting results; optional
from landlab.components import BedrockLandslider  # BedrockLandslider model
from landlab.components import SpaceLargeScaleEroder  # SPACE model
from landlab.components import PriorityFloodFlowRouter

Três componentes do Landlab são indispensáveis para executar o BedrockLandslider: `PriorityFloodFlowRouter`, `SPACE` e `BedrockLandslider`. O roteador Priority‑Flood é particularmente útil quando a grade contém depressões fechadas.

Além desses componentes de processo, utilizaremos utilitários de visualização – por exemplo, `imshow_grid` – para gerar mapas bidimensionais das variáveis do modelo.

### Etapa 2: Definir o domínio do modelo e as condições iniciais

O BedrockLandslider e o SPACE operam em grades raster. Para este exemplo criaremos uma grade retangular com 10 linhas por 25 colunas (total de 250 nós) e espaçamento de 25 m entre nós.

Depois de criada a grade, definiremos as elevações iniciais, profundidade do solo e outras variáveis necessárias.

In [ ]:
# Set grid parameters
num_rows = 50
num_columns = 50
node_spacing = 25.0

# track sediment flux at the node adjacent to the outlet at lower-left
node_next_to_outlet = num_columns + 1

# Instantiate model grid
mg = RasterModelGrid((num_rows, num_columns), node_spacing)
# add field ’topographic elevation’ to the grid
mg.add_zeros("topographic__elevation", at="node")
# set constant random seed for consistent topographic roughness
np.random.seed(seed=5000)

# Create initial model topography:

# add topographic roughness
random_noise = (
    np.random.rand(len(mg.node_y)) / 1000.0
)  # impose topography values on model grid
mg["node"]["topographic__elevation"] += random_noise

# add field 'soil__depth' to the grid
mg.add_zeros("soil__depth", at="node")

# Set 2 m of initial soil depth at core nodes
mg.at_node["soil__depth"][mg.core_nodes] = 1.0  # meters

# Add field 'bedrock__elevation' to the grid
mg.add_zeros("bedrock__elevation", at="node")

# Sum 'soil__depth' and 'bedrock__elevation'
# to yield 'topographic elevation'
mg.at_node["bedrock__elevation"][:] = mg.at_node["topographic__elevation"]
mg.at_node["topographic__elevation"][:] += mg.at_node["soil__depth"]

### Etapa 3: Definir as condições de contorno

É necessário especificar onde água e sedimentos podem sair do domínio. Neste exemplo, deixaremos a borda sul aberta (condição de fluxo livre) e manteremos as demais fronteiras fechadas.

In [ ]:
# Open all model boundary edges
mg.set_closed_boundaries_at_grid_edges(
    bottom_is_closed=False,
    left_is_closed=False,
    right_is_closed=False,
    top_is_closed=False,
)

### Etapa 4: Inicializar o roteador de fluxo e o SPACE

Como na maioria dos componentes do Landlab, criaremos instâncias de `PriorityFloodFlowRouter` e `SPACE`, ajustando os parâmetros relevantes. Nesta etapa ainda não ativaremos o BedrockLandslider.

In [ ]:
# Instantiate flow router
fr = PriorityFloodFlowRouter(mg, flow_metric="D8", suppress_out=True)

# Instantiate SPACE model with chosen parameters
sp = SpaceLargeScaleEroder(
    mg,
    K_sed=2.5e-5,
    K_br=2.5e-5,
    F_f=0.0,
    phi=0.0,
    H_star=1.0,
    v_s=1,
    m_sp=0.5,
    n_sp=1.0,
    sp_crit_sed=0,
    sp_crit_br=0,
)

### Etapa 5: Executar o loop de tempo para desenvolver uma paisagem **sem** atividade de deslizamentos

O componente SPACE calcula o entranhamento e a deposição de sedimentos; ele será executado dentro de um loop que avança até o tempo de modelo atingir o valor definido.

In [ ]:
# Set model timestep
timestep = 1e3  # years

# Set elapsed time to zero
elapsed_time = 0.0  # years

# Set timestep count to zero
count = 0

# Set model run time
run_time = 5e5  # years

# Array to save sediment flux values
sed_flux = np.zeros(int(run_time // timestep))

# Uplift rate in m/yr
U = 1e-3

cmap = copy.copy(mpl.colormaps["terrain"])

while elapsed_time < run_time:  # time units of years
    # Insert uplift at core nodes
    mg.at_node["bedrock__elevation"][mg.core_nodes] += U * timestep
    mg.at_node["topographic__elevation"][:] = (
        mg.at_node["bedrock__elevation"] + mg.at_node["soil__depth"]
    )

    # Run the flow router
    fr.run_one_step()

    # Run SPACE for one time step
    sp.run_one_step(dt=timestep)

    # Add to value of elapsed time
    elapsed_time += timestep

    if np.mod(elapsed_time, 1e5) == 0:
        print("%.2f of model run completed" % (elapsed_time / run_time))
        imshow_grid(
            mg, "topographic__elevation", cmap=cmap, colorbar_label="Elevation (m)"
        )
        plt.show()

## Visualização dos resultados

### Topografia e profundidade do solo

In [ ]:
cmap = copy.copy(mpl.colormaps["terrain"])
# Show DEM draped over the shaded topographic relief
imshowhs_grid(
    mg,
    "topographic__elevation",
    var_name="Topo",
    var_units=r"m",
    grid_units=("m", "m"),
    cmap=cmap,
    ticks_km=False,
)
plt.show()
# Show Soil thickness draped over the shaded topographic relief
cmap = copy.copy(mpl.colormaps["pink"])
imshowhs_grid(
    mg,
    "topographic__elevation",
    drape1=mg.at_node["soil__depth"],
    plot_type="Drape1",
    var_name="Soil",
    var_units=r"m",
    grid_units=("m", "m"),
    cmap=cmap,
    ticks_km=False,
)

z_before_LS = np.array(mg["node"]["topographic__elevation"])

### Etapa 6: Inicializar `PriorityFloodFlowRouter`, `SPACE` e `BedrockLandslider`

`BedrockLandslider` é implementado como uma classe Python. Aqui instanciamos essa classe e definimos os parâmetros necessários. Para calcular o *run‑out* dos deslizamentos utilizamos um esquema de deposição não local baseado em inclinação e drenagem.

In [ ]:
# Instantiate flow router, with additional multiple flow director for hillslopes
fr = PriorityFloodFlowRouter(
    mg,
    flow_metric="D8",
    separate_hill_flow=True,
    hill_flow_metric="Quinn",
    update_hill_flow_instantaneous=True,
)

# Instantiate SPACE model with chosen parameters
hy = BedrockLandslider(
    mg,
    angle_int_frict=0.4,
    cohesion_eff=1e3,
    landslides_return_time=1000,
    landslides_on_boundary_nodes=False,
)

### Etapa 7: Executar o loop de tempo por 200 anos para desenvolver uma paisagem **com** atividade de deslizamentos

Assim como antes, aplicaremos soerguimento, rotearemos o fluxo e executaremos o SPACE. Entretanto, agora o BedrockLandslider estará ativo e os deslizamentos serão simulados estocasticamente.

In [ ]:
# Reset elevation back to elevation simulated without landslides to test various landslide configuration settings

mg["node"]["topographic__elevation"][:] = z_before_LS
timestep = 20  # years
landslides_size_all_steps = []

for i in range(10):
    # Insert uplift at core nodes
    mg.at_node["bedrock__elevation"][mg.core_nodes] += U * timestep
    mg.at_node["topographic__elevation"][:] = (
        mg.at_node["bedrock__elevation"] + mg.at_node["soil__depth"]
    )

    # Run the flow router
    fr.run_one_step()

    # Run SPACE for one time step
    sp.run_one_step(dt=timestep)

    # Run BedrockLandslider for one time step
    hy.run_one_step(dt=timestep)

    # Store landslide sizes of current time step into general ls_size list
    landslides_size_all_steps = np.append(landslides_size_all_steps, hy.landslides_size)

## Visualização dos resultados
### Distribuição magnitude–frequência dos deslizamentos simulados em 200 anos

In [ ]:
LS_size = landslides_size_all_steps * mg.dx**2
counts, bins = np.histogram(np.log10(LS_size), 10)
plt.hist(np.log10(LS_size), log=True, bins=bins, density=True)
plt.xlabel("log10 LS Area, m2")
plt.ylabel("Landslide frequency")

### Localização dos deslizamentos na última iteração do modelo

Vamos plotar os deslizamentos resultantes.

In [ ]:
# Landslide Erosion
cmap = copy.copy(mpl.colormaps["hot_r"])
imshow_grid(
    mg,
    np.sqrt(mg.at_node["landslide__erosion"]),
    colorbar_label="SQRT( Landslide erosion, m) ",
    cmap=cmap,
)
plt.show()

# Landslide Deposition
cmap = copy.copy(mpl.colormaps["winter_r"])
imshow_grid(
    mg,
    np.sqrt(mg.at_node["landslide__deposition"]),
    colorbar_label="SQRT( Landslide deposition, m) ",
    cmap=cmap,
)
plt.show()